In [ ]:
import keras
import cv2
import numpy as np
from keras.utils import img_to_array
from playsound import playsound
from threading import Thread
import time

def alert():
    while True:
        playsound('data/alarm.mpeg')

classes = ['Closed', 'Open']
face_cascade = cv2.CascadeClassifier(
    r"haarcascade/haarcascade_frontalface_default.xml")
left_eye_cascade = cv2.CascadeClassifier(
    r"haarcascade/haarcascade_lefteye_2splits.xml")
right_eye_cascade = cv2.CascadeClassifier(
    r"haarcascade/haarcascade_righteye_2splits.xml")
cap = cv2.VideoCapture(0)
model = keras.models.load_model(
    r"drowiness_new6(1).h5")
count = 0
alarm_on = False
status1 = ''
status2 = ''
# Initialize the variable corresponding to the FPS calculation
prev_frame_time = 0
new_frame_time = 0

while True:
    _, frame = cap.read()
    height = frame.shape[0]
    framegray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(framegray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = framegray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        left_eye = left_eye_cascade.detectMultiScale(roi_gray)
        right_eye = right_eye_cascade.detectMultiScale(roi_gray)
        for (x1, y1, w1, h1) in left_eye:
            cv2.rectangle(roi_color, (x1, y1),
                          (x1 + w1, y1 + h1), (0, 255, 0), 1)
            eye1 = roi_color[y1:y1+h1, x1:x1+w1]
            eye1 = cv2.resize(eye1, (145, 145))
            eye1 = eye1.astype('float') / 255.0
            eye1 = img_to_array(eye1)
            eye1 = np.expand_dims(eye1, axis=0)
            pred1 = model.predict(eye1)
            status1 = np.argmax(pred1)
            break

        for (x2, y2, w2, h2) in right_eye:
            cv2.rectangle(roi_color, (x2, y2),
                          (x2 + w2, y2 + h2), (0, 255, 0), 1)
            eye2 = roi_color[y2:y2 + h2, x2:x2 + w2]
            eye2 = cv2.resize(eye2, (145, 145))
            eye2 = eye2.astype('float') / 255.0
            eye2 = img_to_array(eye2)
            eye2 = np.expand_dims(eye2, axis=0)
            pred2 = model.predict(eye2)
            status2 = np.argmax(pred2)
            break

        # If the eyes are closed, start counting
        if status1 == 2 and status2 == 2:
            count += 1
            cv2.putText(frame, "Eyes Closed, Frame count: " + str(count),
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
            # if eyes are closed for 10 consecutive frames, start the alarm
            if count >= 3:
                cv2.putText(frame, "Drowsiness Alert!!!", (100, height-20),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                if not alarm_on:
                    alarm_on = True
                    # play the alarm sound in a new thread
                    t = Thread(target=alert)
                    t.daemon = True
                    t.start()
        else:
            cv2.putText(frame, "Eyes Open", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
            count = 0
            alarm_on = False
    # Caculate FPS
    new_frame_time = time.time()
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    # Display FPS value on video stream
    cv2.putText(frame, f"FPS:{int(fps)}", (510, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

    cv2.imshow("Drowsiness Detector", frame)

    key = cv2.waitKey(1) & 0xFF
    # Check if user presses "q" key or ESC button, exit while loop
    if key == ord("q") or key == 27:
        break

cap.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 133ms/step


1/1 [==============================] - 0s 173ms/step



    Error 259 for command:
        play data/alarm.mpeg wait
    The driver cannot recognize the specified command parameter.
Exception in thread Thread-5 (alert):
Traceback (most recent call last):
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jnaks\AppData\Local\Temp\ipykernel_14220\2131957361.py", line 11, in alert
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 73, in _playsoundWin
    winCommand(u'play {}{}'.format(sound, ' wait' if block else ''))
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(exceptionMessage)
playsound.PlaysoundException: 
    Error 259 for command:
        play data/alarm.mpeg wait
    The driver cannot recognize the specified command parameter.


1/1 [==============================] - 0s 173ms/step



    Error 263 for command:
        open data/alarm.mpeg
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close data/alarm.mpeg
    The specified device is not open or is not recognized by MCI.
Failed to close the file: data/alarm.mpeg
Exception in thread Thread-6 (alert):
Traceback (most recent call last):
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jnaks\AppData\Local\Temp\ipykernel_14220\2131957361.py", line 11, in alert
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(exceptionMessage)
playsound.PlaysoundException: 
    Error 263 for command:
      

1/1 [==============================] - 0s 47ms/step



    Error 263 for command:
        open data/alarm.mpeg
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close data/alarm.mpeg
    The specified device is not open or is not recognized by MCI.
Failed to close the file: data/alarm.mpeg
Exception in thread Thread-7 (alert):
Traceback (most recent call last):
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jnaks\AppData\Local\Temp\ipykernel_14220\2131957361.py", line 11, in alert
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(exceptionMessage)
playsound.PlaysoundException: 
    Error 263 for command:
      

1/1 [==============================] - 0s 53ms/step



    Error 263 for command:
        close data/alarm.mpeg
    The specified device is not open or is not recognized by MCI.
Failed to close the file: data/alarm.mpeg

    Error 263 for command:
        open data/alarm.mpeg
    The specified device is not open or is not recognized by MCI.
Exception in thread Thread-8 (alert):
Traceback (most recent call last):
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\jnaks\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jnaks\AppData\Local\Temp\ipykernel_14220\2131957361.py", line 11, in alert
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "C:\Users\jnaks\anaconda3\Lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(exceptionMessage)
playsound.PlaysoundException: 
    Error 263 for command:
      

1/1 [==============================] - 0s 147ms/step


In [ ]:
pip install keras


In [5]:
pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2

   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ------------------------------ 62.7/300.9 MB 168.6 kB/s eta 0:23:34
   -------- ----------------------------

In [17]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install playsound


In [22]:
pip install threaded


Note: you may need to restart the kernel to use updated packages.
